<h1><center><b>Primjena modela strojnog učenja za predviđanje potražnje</b></center></h1>


Ova bilježnica napravljena je u svrhu natjecanja za dodjelu GDi for STEM nagrade za izvrsnost za 2022. godinu koja je raspisana za  studentice diplomskog studija Sveučilišta u Zagrebu Fakulteta elektrotehnike i računarstva. Zadatak natjecanja je napraviti analizu podatkovnog skupa preuzetog iz GDi Ensemble sustava te pomoću strojnog učenja riješiti predviđanja potražnje. 

SADRŽAJ
1. <a href=#predobrada>Predobrada podataka</a>
2. <a href=#analiza>Analiza podataka</a>
3. <a href=#inzenjerstvo>Inženjerstvo značajki</a>
4. <a href=#odabir>Odabir modela</a>
5. <a href=#ucenje>Učenje modela</a>
6. <a href=#predvidanje>Predviđanje modela</a>
5. <a href=#RMSE>RMSE</a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# analiza podataka
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelBinarizer
from statsmodels.tsa.seasonal import seasonal_decompose
import seaborn as sns
import plotly.express as px
import seaborn as sns

# inženjerstvo značajki
import holidays
hr_holidays = holidays.country_holidays('HR')

# učenje modela
import xgboost as xgb
from xgboost import plot_importance, plot_tree


MIN_DATE = '2021-06-22'
MAX_DATE =  '2022-07-19'

In [ ]:
# učitavanje podataka
data = pd.read_csv('./dataset/task_data.csv')
print(data.head())

## 1. Predobrada podataka <a name='predobrada' />

In [ ]:
print(type(data['Date']))

# tip podataka stupca 'Date' pretvoren u DateTime tip podataka
data['Date'] = pd.to_datetime(data['Date'])

# stupac 'Date' postavljen kao indeks tablice - treba za umetanje datuma
data = data.set_index('Date')
data

In [ ]:
# provjera ima li prethodnih nula u tablici
print(data[data['Quantity'] == 0])

# provjera ima li nedostajućih vrijednosti u tablici
print(data[data['Quantity'].isna()])

In [ ]:
# umetanje nedostajućih datuma 
date_list = pd.date_range(MIN_DATE, MAX_DATE, freq="D")

def reindex_by_date(data):
    i = data['ItemId'].unique()[0]
    data = data.reindex(date_list)

    data['ItemId'] = data['ItemId'].fillna(i).astype(int)
    data['Quantity'] = data['Quantity'].fillna(0).astype(int)

    return data

data = data.groupby('ItemId').apply(reindex_by_date).reset_index(0, drop=True)
data.index.rename('Date', inplace='True')

data.head()

## 2. Analiza podataka <a name='analiza' />

In [ ]:
# prikaz količine naručenog proizvoda u ovisnosti o datumu
for item in set(data['ItemId']):
    f = plt.figure()
    f.set_figwidth(50)
    f.set_figheight(10)
    plt.title('ID = ' + str(item))
    plt.xlabel('Time (days)')
    plt.ylabel('Quantity')
    plt.plot(data[data['ItemId'] == item]['Quantity'])
    #plt.savefig('./time_series/ItemId = ' + str(item) + '.png') 
    plt.figure()

In [ ]:
# moving average
for i in data['ItemId'].unique():
    subset = data[data['ItemId'] == i].copy()
    subset['moving_average'] = subset['Quantity'].rolling(30).mean()
    fig = px.line(subset, x=subset.index, y=["Quantity","moving_average"], title = 'item id = ' + str(i))
    fig.show()

In [ ]:
# analiza podataka dekompozicijom na sezonalnost, trend i rezidual
component_names = ['observed', 'seasonal', 'trend', 'resid']

for item in set(data['ItemId']) :
    fig, axes = plt.subplots(2,2, figsize=(30,10))
    fig.suptitle('ItemId = ' + str(item), fontsize=15)
    decomposed = seasonal_decompose(data[data['ItemId'] == item]['Quantity'], model='additive')
    components = [decomposed.observed, decomposed.seasonal, decomposed.trend, decomposed.resid]
    i = 0
    for ax, component in zip(axes.flat, components):
        ax.plot(component)
        ax.set_xlabel('Time')
        ax.set_ylabel(component_names[i])
        
        i = i+1
    #plt.savefig('./seasonal_decompose/ItemId = ' + str(item) + '.png')

In [ ]:
# skup podataka za učenje
train_data = data['2021-06-26':'2022-06-19']

# skup podataka za evaluaciju
test_data = data['2022-06-20':'2022-07-19']

In [ ]:
# prikaz skupa za učenje i skupa za validaciju
for item in set(test_data['ItemId']):
    plt.plot(test_data[test_data['ItemId'] == item]['Quantity'].index, test_data[test_data['ItemId'] == item]['Quantity'])
    plt.plot(train_data[train_data['ItemId'] == item]['Quantity'].index, train_data[train_data['ItemId'] == item]['Quantity'])
    plt.figure()


## 3. Inženjerstvo značajki <a name='inzenjerstvo' />

In [ ]:
def create_features(data, is_train=False) :
    data = data.copy()
    data['dayofweek'] = data.index.dayofweek
    data['quarter'] = data.index.quarter
    data['month'] = data.index.month
    data['year'] = data.index.year
    data['dayofyear'] = data.index.dayofyear
    data['dayofmonth'] = data.index.day
    data['weekofyear'] = data.index.isocalendar().week.astype('int64')
    data['is_holiday'] = pd.DataFrame(index=data.index, data={"is_holiday": [date in hr_holidays for date in data.index]})
    
    if is_train :
        data['lag1'] = data.groupby(['ItemId'])['Quantity'].shift(1)
        data['lag2'] = data.groupby(['ItemId'])['Quantity'].shift(2)
        data['lag3'] = data.groupby(['ItemId'])['Quantity'].shift(3)
        data['lag7'] = data.groupby(['ItemId'])['Quantity'].shift(7)
        
        avg = data.groupby(['ItemId'])['Quantity'].rolling(30).mean().shift(1).reset_index()['Quantity']
        avg.index = data.index
        data['avg_month'] = avg
        
        avg = data.groupby(['ItemId'])['Quantity'].rolling(7).mean().shift(1).reset_index()['Quantity']
        avg.index = data.index
        data['avg_week'] = avg

        avg = data.groupby(['ItemId', 'dayofweek'])['Quantity'].rolling(20, min_periods=1).mean().shift(1).reset_index()['Quantity']
        avg.index = data.index
        data['avg_dayofweek'] = avg
        
    return data

In [ ]:
train_data = create_features(train_data, True)
test_data = create_features(test_data)

In [ ]:
import seaborn as sns

plt.figure(figsize=(16, 6))
heatmap = sns.heatmap(train_data.corr(), vmin=-1, vmax=1, annot=True)
heatmap.set_title('Correlation Heatmap', fontdict={'fontsize':12}, pad=12);

In [ ]:
train_data = train_data.drop(['quarter', 'month', 'year', 'dayofyear', 'dayofmonth', 'weekofyear', 'is_holiday'], axis=1)
test_data = test_data.drop(['quarter', 'month', 'year', 'dayofyear', 'dayofmonth', 'weekofyear', 'is_holiday'], axis=1)

## 4. Odabir modela <a name='odabir' />

In [ ]:
def transform_row(X_f, row) :
    row = pd.DataFrame([row.values], columns = row.index.values, index=[pd.to_datetime(row.name)])
    row = row.drop('Quantity', axis=1)
    row['lag1'] = X_f.iloc[-1:]['Quantity'].values[0]
    row['lag2'] = X_f.iloc[-2:]['Quantity'].values[0]
    row['lag3'] = X_f.iloc[-3:]['Quantity'].values[0]
    row['lag7'] = X_f.iloc[-7:]['Quantity'].values[0]

    row['avg_month'] = X_f['Quantity'].tail(30).mean()
    row['avg_week'] = X_f['Quantity'].tail(7).mean()
    row['avg_dayofweek'] = X_f[X_f['dayofweek']==row['dayofweek'].values[0]]['Quantity'].tail(20).mean()
    return row
    

In [ ]:
# grid search

max_depth = [2, 5, 7]
n_estimators = [10, 100, 250]
eta = [0.2, 0.7]

 # skup podataka za učenje
train = data['2021-06-26':'2022-05-20']
train = create_features(train, is_train=True)
X_train = train.drop('Quantity',axis=1)
y_train = train['Quantity']

# skup podataka za validaciju
val = data['2022-05-21':'2022-06-19']
val = create_features(val)
X_val = val.drop('Quantity',axis=1)
y_val = val['Quantity']

for e in eta :
    for d in max_depth :
        for n in n_estimators :

            reg = xgb.XGBRegressor(eta = e, max_depth = d, n_estimators = n)
            reg.fit(X_train, y_train, verbose=False)

            predicted = []

            X = train.copy() # kopiraj skup za učenje u X
            for item in set(train['ItemId']) : # iteriraj po proizvodima
                i = 0

                items = val[val['ItemId'] == item]
                for index, row in items.iterrows(): 
                    X_f = X[X['ItemId'] == item] 
                    row = transform_row(X_f, row) # izračunaj lag i moving average značajke na temelju X_f

                    y_pred = reg.predict(row) # predviđanje primjera skupa za validaciju
                    predicted.append(y_pred)
                    row['Quantity'] = y_pred

                    X = pd.concat([X,row]) # ažuriraj skup podataka x

            score = mean_squared_error(predicted, y_val.values, squared=False)
            print('hyperparameters= {eta = ' + str(e) + ', max_depth= ' + str(d) + ', n_estimators= ' + str(n)+ '}')
            print(f'RMSE Score on Validation set: {score:0.2f}\n')

## 5. Učenje modela <a name='ucenje' />

In [ ]:
TARGET = 'Quantity'

y_train = train_data[TARGET]
X_train = train_data.drop(TARGET, axis=1)

y_test = test_data[TARGET]
X_test = test_data.drop(TARGET, axis=1)

In [ ]:
X_train = train_data.drop(TARGET, axis=1)
X_test = test_data.drop(TARGET, axis=1)

In [ ]:
# učenje odabranog modela na skupu podataka za učenje
reg = xgb.XGBRegressor(eta=0.7,  n_estimators=250, max_depth=2)
reg.fit(X_train, y_train) 

In [ ]:
## prikaz predviđene količine i stvarne količine naručenog proizvoda na skupu podataka za učenje
predicted = reg.predict(X_train)

predicted = pd.DataFrame(predicted)

predicted['ItemId'] = X_train.reset_index()['ItemId']
for item in set(train_data['ItemId']):
    plt.plot(train_data[train_data['ItemId'] == item]['Quantity'])
    plt.plot(X_train[train_data['ItemId'] == item].index, predicted[predicted['ItemId'] == item][0])
    plt.figure()

## 6. Predviđanje modela <a name='predvidanje' />

In [ ]:
# predviđanje modela 
import warnings
warnings.filterwarnings("ignore")

X = X_train.copy()
X['Quantity'] = y_train
y = y_train.copy()

predicted = []
for item in set(test_data['ItemId']) :
    items = test_data[test_data['ItemId'] == item]

    for index, row in items.iterrows():
        X_f = X[X['ItemId'] == item]
        row = transform_row(X_f, row)
                
        y_pred = reg.predict(row)
        predicted.append(y_pred)
        row['Quantity'] = y_pred
    
        X = pd.concat([X,row])

In [ ]:
# prikaz količine naručenog proizvoda iz skupa za validaciju i predviđene količine naručenog proizvoda
predicted = pd.DataFrame(predicted)
predicted['ItemId'] = X_test.reset_index()['ItemId']

for item in set(test_data['ItemId']):
    f = plt.figure()
    f.set_figwidth(30)
    f.set_figheight(10)
    plt.title('ID = ' + str(item))
    plt.xlabel('Time (days)')
    plt.ylabel('Quantity')
    plt.plot(test_data[test_data['ItemId'] == item]['Quantity'])
    plt.plot(X_test[test_data['ItemId'] == item].index, predicted[predicted['ItemId'] == item][0])
    plt.figure()

In [ ]:
# prikaz važnosti značajki u modelu
_ = plot_importance(reg, height=0.9)

## 7. RMSE <a name='RMSE' />

In [ ]:
# rezultat predviđanja modela na skupu za validaciju
score = mean_squared_error(predicted[0], y_test.values, squared=False)
print(f'RMSE Score on Test set: {score:0.2f}')    